#### load in modules

In [1]:
from tqdm import tqdm
import pandas as pd
import numpy as np
import os

#### read in simulation output

In [2]:
# specify path to simulated data
infile = './results/T20.csv.gz'
# read in simulated data
simulated_data = pd.read_csv(infile, compression='gzip')

#### calculate detection probs

In [3]:
# number of draws for each sampling proportion
num_draws = 200

In [4]:
# specify sampling proportions
sfs = [0.01, 0.05, 0.15, 0.5, 0.9]

In [5]:
# get time range from simulated data
time_range = [t for t in range(simulated_data.cluster_imported_time.min(), simulated_data.cluster_imported_time.max()+1)]

In [6]:
# precompute variables
num_experiments = simulated_data['experiment'].nunique()
true_num_clusters = simulated_data['cluster'].nunique()
infected_data = simulated_data[simulated_data['state'] >= 1]  # Filter once for infected individuals

# prepare dictionaries for detection numbers
sf_exp_time_detection_nums = {}

for sf in sfs:
    print(f'sampling proportion: {sf}')
    exp_time_detection_nums = {time: [] for time in time_range}

    for exp_i in tqdm(range(num_experiments)):
        # filter data for the current experiment
        df_exp = infected_data[infected_data['experiment'] == exp_i]
        num_sample = int(sf * len(df_exp))

        for _ in range(num_draws):
            # sample individuals
            sampled_infected = df_exp.sample(n=num_sample, replace=False)

            # bin clusters by importation time
            time_detection_counts = (
                sampled_infected.groupby('cluster')
                .first()  # get the first record of each cluster
                .groupby('cluster_imported_time')
                .size()
                .to_dict()
            )

            # store detection numbers for each time
            for time in time_range:
                exp_time_detection_nums[time].append(time_detection_counts.get(time, 0))

    # store results for the sampling proportion
    sf_exp_time_detection_nums[sf] = exp_time_detection_nums


sampling proportion: 0.01


100%|██████████| 30/30 [00:02<00:00, 10.00it/s]


sampling proportion: 0.05


100%|██████████| 30/30 [00:02<00:00, 11.50it/s]


sampling proportion: 0.15


100%|██████████| 30/30 [00:02<00:00, 11.34it/s]


sampling proportion: 0.5


100%|██████████| 30/30 [00:02<00:00, 10.87it/s]


sampling proportion: 0.9


100%|██████████| 30/30 [00:02<00:00, 10.82it/s]


#### get medians and interquartile ranges

In [24]:
temporal_median = { sf: { time: np.median(sf_exp_time_detection_nums[sf][time])
                         for time in sf_exp_time_detection_nums[sf].keys() } for sf in sfs }
temporal_lws_95CI = { sf: { time: np.percentile(sf_exp_time_detection_nums[sf][time], 2.5)
                           for time in sf_exp_time_detection_nums[sf].keys() } for sf in sfs }
temporal_ups_95CI = { sf: { time: np.percentile(sf_exp_time_detection_nums[sf][time], 97.5)
                           for time in sf_exp_time_detection_nums[sf].keys() } for sf in sfs }

In [25]:
# specify output folder
out_dir = './results/T20'

In [26]:
# export detection numbers by importation time (median, interquartile ranges)
with open(os.path.join(out_dir, 'detection_numbers_by_time.csv'), 'w+') as outfile:
    outfile.write('sf,time,median,lw_95CI,up_95CI\n')
    outfile.write('\n'.join(
        ['%f,%d,%f,%f,%f' % (sf, time, temporal_median[sf][time], temporal_lws_95CI[sf][time], temporal_ups_95CI[sf][time])
         for sf in sfs for time in temporal_median[sf].keys()]))